# export tiles for gain and loss
Based on [ee code](https://code.earthengine.google.com/8423c718d647f15ab3e9c31868489fd4?noload=true)

In [ ]:
import ee
import geemap
%run utils.ipynb

In [ ]:
# Trigger the authentication flow.
# ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate()


Successfully saved authorization token.


In [ ]:
# Initialize the library.
ee.Initialize()

### Data Assets

In [ ]:
## Assets selection
gcs_bucket = 'mangrove_atlas'
version = 'v3'
dataset = f'mangrove_blue_carbon-{version}'
data_year_range = [1996, 2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]
# region for export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# gain
imageCollection = ee.ImageCollection(
    f"projects/global-mangrove-watch/mangrove-properties/{dataset}")

In [ ]:
# styles
color_ramp ='''
    <RasterSymbolizer>
    <ColorMap type="ramp" extended="false" >
      <ColorMapEntry color="#5c4a3d" quantity="400" label="400"/>
      <ColorMapEntry color="#933a06" quantity="800" label="800" />
      <ColorMapEntry color="#b84e17" quantity="1200" label="1200" />
      <ColorMapEntry color="#e68518" quantity="1600" label="1600" />
      <ColorMapEntry color="#eeb66b" quantity="2000" label="2000" />
    </ColorMap>
  </RasterSymbolizer>
'''

### Vizualize the data prior exporting it

In [ ]:
Map = geemap.Map(center=(-5, 39), zoom=12, basemap='HYBRID')

Map.addLayer(imageCollection.first().sldStyle(color_ramp), {}, dataset, True, 0.5)
Map

Map(center=[-5, 39], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

### Export to tiles from GEE to GCS

In [ ]:
# Create a list of tasks to export the map tiles
tasks_list = exportMapTasks(imageCollection, dataset, gcs_bucket, 
                        data_year_range, region, color_ramp, 0, 12)

In [ ]:
# execute tasks in batches due to gee limits
batchExecute(tasks_list)